<a href="https://colab.research.google.com/github/Anson422/Anson-Eva-Valerie/blob/main/Text%20Analysis/Code_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/Anson422/Anson-Eva-Valerie.git #clone the repository to the local machine
!pip install PyPDF2

fatal: destination path 'Anson-Eva-Valerie' already exists and is not an empty directory.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [11]:
#install the necessary libraries
import os
import PyPDF2
from collections import Counter
import pandas as pd

In [13]:
# Define the folder containing the EU energy publications in 2024
EU_Energy_publications_2024 = '/Text Analysis/EU publications about Energy in 2024'

In [14]:
# List of keywords to analyze
keywords = ["gas supply", "LNG", "renewables", "pipeline alternatives"]

# Initialize a counter for keyword frequency
keyword_counts = Counter()